In [3]:
# === ЯЧЕЙКА 1. Импорты и чтение Excel без блокировки файла ===

import pandas as pd
from pathlib import Path

# Укажи путь к файлу — если лежит рядом с ноутбуком, менять не надо
INPUT_FILE = Path("inputs1_wf_sales_opt.xlsx")

if not INPUT_FILE.exists():
    raise FileNotFoundError(f"❌ Файл не найден: {INPUT_FILE}")

# read_excel → файл сразу закрывается → Excel сможет свободно сохранять
sheets = pd.read_excel(INPUT_FILE, sheet_name=None, engine="openpyxl")

# .copy() отрывает датафреймы от файла → исключаем блокировку
df = {name: sh.copy() for name, sh in sheets.items()}

print("✅ Excel загружен, файл НЕ заблокирован.")
print("📄 Листы:", ", ".join(df.keys()))


✅ Excel загружен, файл НЕ заблокирован.
📄 Листы: мэппинг, ограничения_объемы, ограничения_качество, переключатели, добыча, калория_добычи, себестоимость добычи на пе, транспортные расходы до ОФ, мощность оф, расходы на обогащение перем, расходы на обогащение пост, оф_выход, оф_качество, транспортировка до станции отпр


In [4]:
# === ЯЧЕЙКА 2. Множества: ПЕ, ОФ, режимы, ТП, категории, месяцы, VALID_COMBOS ===

from datetime import datetime

# 1) Месяцы и ПЕ из листа "добыча"
df_mining = df["добыча"].copy()
if "ПЕ" not in df_mining.columns:
    raise ValueError("❌ В листе 'добыча' нет колонки 'ПЕ'")

MONTHS = [c for c in df_mining.columns if isinstance(c, (datetime, pd.Timestamp))]
PEs = df_mining["ПЕ"].dropna().astype(str).unique().tolist()

# 2) ТП и категории из "мэппинг"
df_map = df["мэппинг"].copy()
for col in ["ТП", "Категория"]:
    if col not in df_map.columns:
        raise ValueError(f"❌ В листе 'мэппинг' нет колонки '{col}'")

TPs = df_map["ТП"].dropna().astype(str).unique().tolist()
CATEGORIES = df_map["Категория"].dropna().astype(str).unique().tolist()

TP_TO_CATEGORY = {
    tp: str(df_map.loc[df_map["ТП"] == tp, "Категория"].values[0]).strip().lower()
    for tp in TPs
}

CATEGORY_TO_TPS = {}
for cat in CATEGORIES:
    cat_l = cat.strip().lower()
    CATEGORY_TO_TPS[cat_l] = [
        tp for tp, c in TP_TO_CATEGORY.items() if c == cat_l
    ]

# 3) ОФ из "мощность оф"
df_power = df["мощность оф"].copy()
if "ОФ" not in df_power.columns:
    raise ValueError("❌ В листе 'мощность оф' нет колонки 'ОФ'")

OFs = df_power["ОФ"].dropna().astype(str).unique().tolist()

# 4) VALID_COMBOS и режимы из "оф_выход"
df_yield = df["оф_выход"].copy()
required_cols = ["ПЕ", "ОФ", "Режим", "ТП"]
missing = [c for c in required_cols if c not in df_yield.columns]
if missing:
    raise ValueError(f"❌ В листе 'оф_выход' нет колонок: {missing}")

VALID_COMBOS = list(
    df_yield[required_cols]
    .dropna()
    .astype(str)
    .drop_duplicates()
    .itertuples(index=False, name=None)
)

MODES = sorted({mode for (_, _, mode, _) in VALID_COMBOS})

# 5) Маршруты без ТП и вспомогательные словари
ROUTES = sorted({(pe, of, mode) for (pe, of, mode, tp) in VALID_COMBOS})

PE_ROUTES = {pe: [] for pe in PEs}
OF_ROUTES = {of: [] for of in OFs}

for pe, of, mode in ROUTES:
    PE_ROUTES[pe].append((of, mode))
    OF_ROUTES[of].append((pe, mode))

print("✅ Множества сформированы:")
print(f"  ПЕ: {PEs}")
print(f"  ОФ: {OFs}")
print(f"  Режимы: {MODES}")
print(f"  ТП: {TPs}")
print(f"  Категории: {CATEGORIES}")
print(f"  VALID_COMBOS: {len(VALID_COMBOS)}")
print(f"  Месяцев: {len(MONTHS)}")


✅ Множества сформированы:
  ПЕ: ['ш. Ялевского', 'р. Заречный', 'р. Заречный-Северный', 'ш. 7НН', 'ш. Рубана', 'ш. Кирова']
  ОФ: ['ОМ ТЗ-1', 'ОФ Полысаевская', 'ОФ Комсомолец', 'ОФ Кирова Секция 2']
  Режимы: ['1', '2', '3']
  ТП: ['Концентрат 60-130', 'Концентрат 13-60', 'Концентрат +25', 'Концентрат 0-25', 'Концентрат 0-100', 'Отсев 0-13', 'кек и шлам', 'ДР']
  Категории: ['концентрат', 'прочие ТП', 'рядовой']
  VALID_COMBOS: 63
  Месяцев: 12


In [5]:
# === ЯЧЕЙКА 3. Числовые параметры: добыча, мощности, расходы, выход, качество ===

# 1) ДОБЫЧА
MINING = {}
df_mining = df["добыча"]
for _, row in df_mining.iterrows():
    pe = str(row["ПЕ"])
    for m in MONTHS:
        MINING[(pe, m)] = float(row[m])

# 2) МОЩНОСТЬ ОФ
POWER = {}
df_power = df["мощность оф"]
for _, row in df_power.iterrows():
    of = str(row["ОФ"])
    for m in MONTHS:
        POWER[(of, m)] = float(row[m])

# 3) ТРАНСПОРТ ПЕ → ОФ
COST_PE_TO_OF = {}
df_tr = df["транспортные расходы до ОФ"]
for _, row in df_tr.iterrows():
    pe = str(row["ПЕ"])
    of = str(row["ОФ"])
    for m in MONTHS:
        COST_PE_TO_OF[(pe, of, m)] = float(row[m])

# 4) ПЕРЕМЕННЫЕ РАСХОДЫ ОФ
VAR_ENRICH = {}
df_var = df["расходы на обогащение перем"]
for _, row in df_var.iterrows():
    of = str(row["ОФ"])
    for m in MONTHS:
        VAR_ENRICH[(of, m)] = float(row[m])

# 5) ПОСТОЯННЫЕ РАСХОДЫ ОФ
FIX_ENRICH = {}
df_fix = df["расходы на обогащение пост"]
for _, row in df_fix.iterrows():
    of = str(row["ОФ"])
    for m in MONTHS:
        FIX_ENRICH[(of, m)] = float(row[m])

# 6) ВЫХОД ТП (%)
YIELD = {}
df_y = df["оф_выход"]
for _, row in df_y.iterrows():
    pe = str(row["ПЕ"])
    of = str(row["ОФ"])
    mode = str(row["Режим"])
    tp = str(row["ТП"])
    for m in MONTHS:
        YIELD[(pe, of, mode, tp, m)] = float(row[m])

# 7) КАЧЕСТВО ТП
QUALITY = {}
df_q = df["оф_качество"]
for _, row in df_q.iterrows():
    pe = str(row["ПЕ"])
    of = str(row["ОФ"])
    mode = str(row["Режим"])
    tp = str(row["ТП"])
    for m in MONTHS:
        QUALITY[(pe, of, mode, tp, m)] = float(row[m])

print("✅ Числовые параметры успешно считаны:")
print(f"  MINING: {len(MINING)}")
print(f"  POWER: {len(POWER)}")
print(f"  COST_PE_TO_OF: {len(COST_PE_TO_OF)}")
print(f"  VAR_ENRICH: {len(VAR_ENRICH)}")
print(f"  FIX_ENRICH: {len(FIX_ENRICH)}")
print(f"  YIELD: {len(YIELD)}")
print(f"  QUALITY: {len(QUALITY)}")


✅ Числовые параметры успешно считаны:
  MINING: 72
  POWER: 48
  COST_PE_TO_OF: 108
  VAR_ENRICH: 48
  FIX_ENRICH: 48
  YIELD: 756
  QUALITY: 756


In [6]:
# === ЯЧЕЙКА 4. Переключатели и параметры оптимизации ===

# 1) ПЕРЕКЛЮЧАТЕЛИ
df_sw = df["переключатели"].copy()

if not {"ограничение", "1-вкл/_0-выкл"}.issubset(df_sw.columns):
    raise ValueError("❌ В листе 'переключатели' должны быть колонки: 'ограничение', '1-вкл/_0-выкл'")

SWITCHES = {
    str(row["ограничение"]).strip(): int(row["1-вкл/_0-выкл"])
    for _, row in df_sw.iterrows()
}

# 2) ПАРАМЕТРЫ ОПТИМИЗАЦИИ ИЗ ОГРАНИЧЕНИЙ

col_pr = "Приоритет верхенго уровня (0/1)"
col_subj = "субъект оптимизации (категория тп/все)"
col_goal = "цель оптимизации (min/max/to)"
col_target = "значение (если выбрано to)"

df_vol = df["ограничения_объемы"].copy()
df_qual = df["ограничения_качество"].copy()

OPTIMIZE_VOLUME = int(df_vol.loc[0, col_pr])  # теперь только 0 или 1
OPTIMIZE_QUALITY = int(df_qual.loc[0, col_pr])

SUBJECT_VOLUME = str(df_vol.loc[0, col_subj]).strip().lower()
SUBJECT_QUALITY = str(df_qual.loc[0, col_subj]).strip().lower()

GOAL_VOLUME = str(df_vol.loc[0, col_goal]).strip().lower()
GOAL_QUALITY = str(df_qual.loc[0, col_goal]).strip().lower()

TARGET_VOLUME = df_vol.loc[0, col_target]
TARGET_QUALITY = df_qual.loc[0, col_target]

# 3) Валидация

if OPTIMIZE_VOLUME not in (0, 1) or OPTIMIZE_QUALITY not in (0, 1):
    raise ValueError("❌ Приоритеты должны быть 0 или 1.")

if OPTIMIZE_VOLUME == 1 and OPTIMIZE_QUALITY == 1:
    raise ValueError("❌ Нельзя одновременно оптимизировать объём и качество — выбери одно.")

print("✅ Параметры оптимизации загружены:")
print("SWITCHES:", SWITCHES)
print(f"OPTIMIZE_VOLUME = {OPTIMIZE_VOLUME}, SUBJECT_VOLUME = '{SUBJECT_VOLUME}', GOAL_VOLUME = '{GOAL_VOLUME}', TARGET_VOLUME = {TARGET_VOLUME}")
print(f"OPTIMIZE_QUALITY = {OPTIMIZE_QUALITY}, SUBJECT_QUALITY = '{SUBJECT_QUALITY}', GOAL_QUALITY = '{GOAL_QUALITY}', TARGET_QUALITY = {TARGET_QUALITY}")


✅ Параметры оптимизации загружены:
SWITCHES: {'минимизация_себестоимости': 1, 'учитывать_мощность_оф': 1, 'полностью_загружать_фабрики': 1, 'учитывать_логистические_ограничения': 0, 'учитывать_мощность_портов': 0, 'поставки_на_внутренний_рынок_в_первую_очередь': 0, 'строгие_контракты_на_внутреннем_рынке': 0, 'ограничение_калорийности_продукта_на_вр': 0}
OPTIMIZE_VOLUME = 0, SUBJECT_VOLUME = 'концентрат', GOAL_VOLUME = 'max', TARGET_VOLUME = nan
OPTIMIZE_QUALITY = 1, SUBJECT_QUALITY = 'концентрат', GOAL_QUALITY = 'max', TARGET_QUALITY = nan


In [7]:
# === ЯЧЕЙКА 5. Создание модели и переменных ===

import pulp

model = pulp.LpProblem("Factory_Optimization", pulp.LpMaximize)

# 1) Уголь, который отправляем на фабрики
x_in = pulp.LpVariable.dicts(
    "x_in",
    ((pe, of, mode, m) for (pe, of, mode) in ROUTES for m in MONTHS),
    lowBound=0
)

# 2) Готовая продукция
x_tp = pulp.LpVariable.dicts(
    "x_tp",
    ((pe, of, mode, tp, m) for (pe, of, mode, tp) in VALID_COMBOS for m in MONTHS),
    lowBound=0
)

# 3) Остатки угля на ПЕ
stock_pe = pulp.LpVariable.dicts(
    "stock_pe",
    ((pe, m) for pe in PEs for m in MONTHS),
    lowBound=0
)

print("✅ Переменные успешно созданы:")
print(f"  x_in: {len(x_in)}")
print(f"  x_tp: {len(x_tp)}")
print(f"  stock_pe: {len(stock_pe)}")


✅ Переменные успешно созданы:
  x_in: 360
  x_tp: 756
  stock_pe: 72


In [8]:
# === ЯЧЕЙКА 6. Ограничения баланса, выхода и мощностей ===

# 1) Баланс добычи: отправили + на складе = добыли
for pe in PEs:
    for m in MONTHS:
        model += (
            pulp.lpSum(x_in[(pe, of, mode, m)] for (of, mode) in PE_ROUTES[pe])
            + stock_pe[(pe, m)]
            == MINING[(pe, m)]
        ), f"mining_balance_{pe}_{m}"

# 2) Выход продуктов: ТП = уголь * выход
for (pe, of, mode, tp) in VALID_COMBOS:
    for m in MONTHS:
        model += (
            x_tp[(pe, of, mode, tp, m)]
            == x_in[(pe, of, mode, m)] * YIELD[(pe, of, mode, tp, m)]
        ), f"yield_relation_{pe}_{of}_{mode}_{tp}_{m}"

# 3) Ограничение максимальной мощности фабрик
if SWITCHES.get("учитывать_мощность_оф", 0) == 1:
    for of in OFs:
        for m in MONTHS:
            model += (
                pulp.lpSum(
                    x_in[(pe, of, mode, m)]
                    for (pe, mode) in OF_ROUTES[of]
                )
                <= POWER[(of, m)]
            ), f"capacity_{of}_{m}"

# 4) Умная минимальная загрузка фабрик ≥90%
if SWITCHES.get("полностью_загружать_фабрики", 0) == 1:
    for of in OFs:
        for m in MONTHS:
            total_available = sum(MINING[(pe, m)] for pe in PEs)

            if total_available >= POWER[(of, m)]:
                model += (
                    pulp.lpSum(
                        x_in[(pe, of, mode, m)]
                        for (pe, mode) in OF_ROUTES[of]
                    )
                    >= 0.9 * POWER[(of, m)]
                ), f"min_load_{of}_{m}"

print("✅ Ограничения баланса, выхода и мощностей добавлены.")



✅ Ограничения баланса, выхода и мощностей добавлены.


In [9]:
# === ЯЧЕЙКА 7. Агрегаторы по выбранным категориям для объёма и качества ===

import math
from collections import defaultdict

def build_category_aggregates(category_name: str):
    """
    Возвращает:
      VOL[(pe, m)] — объём ТП выбранной категории,
      EN[(pe, m)]  — суммарная 'энергия' (объём * калорийность)
    для каждой ПЕ и месяца.
    """
    cat = category_name.strip().lower()
    if cat not in CATEGORY_TO_TPS:
        raise ValueError(f"❌ Категория '{category_name}' не найдена в мэппинге.")

    tps_cat = set(CATEGORY_TO_TPS[cat])

    VOL = {}
    EN = {}

    for pe in PEs:
        for m in MONTHS:
            VOL[(pe, m)] = pulp.lpSum(
                x_tp[(pe2, of, mode, tp, m)]
                for (pe2, of, mode, tp) in VALID_COMBOS
                if pe2 == pe and tp in tps_cat
            )
            EN[(pe, m)] = pulp.lpSum(
                x_tp[(pe2, of, mode, tp, m)] * QUALITY[(pe2, of, mode, tp, m)]
                for (pe2, of, mode, tp) in VALID_COMBOS
                if pe2 == pe and tp in tps_cat
            )

    return VOL, EN


# Агрегаторы для объёма (если включен OPTIMIZE_VOLUME)
VOL_FOR_VOLUME = {}
EN_DUMMY_FOR_VOLUME = {}
if OPTIMIZE_VOLUME == 1:
    VOL_FOR_VOLUME, EN_DUMMY_FOR_VOLUME = build_category_aggregates(SUBJECT_VOLUME)

# Агрегаторы для качества (если включен OPTIMIZE_QUALITY)
VOL_FOR_QUALITY = {}
EN_FOR_QUALITY = {}
if OPTIMIZE_QUALITY == 1:
    VOL_FOR_QUALITY, EN_FOR_QUALITY = build_category_aggregates(SUBJECT_QUALITY)

print("✅ Агрегаторы по категориям построены.")
if OPTIMIZE_VOLUME == 1:
    print(f"  Для объёма используем категорию: '{SUBJECT_VOLUME}'")
if OPTIMIZE_QUALITY == 1:
    print(f"  Для качества используем категорию: '{SUBJECT_QUALITY}'")


✅ Агрегаторы по категориям построены.
  Для качества используем категорию: 'концентрат'


In [10]:
# === ЯЧЕЙКА 8. Целевая функция — выбор режима по качеству/объёму ===


BIG = 10**9  # безопасное большое число

# ---- бинарные переменные выбора режима ----
y = pulp.LpVariable.dicts(
    "y",
    [(pe, of, mode, m) for pe in PEs for (of, mode) in PE_ROUTES[pe] for m in MONTHS],
    cat="Binary"
)

# ---- связка x_in с выбором режима ----
for pe in PEs:
    for (of, mode) in PE_ROUTES[pe]:
        for m in MONTHS:
            model += (
                x_in[(pe, of, mode, m)] <= BIG * y[(pe, of, mode, m)],
                f"bind_x_to_y_{pe}_{of}_{mode}_{m}"
            )

# ---- на один ПЕ в месяц максимум один режим ----
for pe in PEs:
    for m in MONTHS:
        model += (
            pulp.lpSum(y[(pe, of, mode, m)] for (of, mode) in PE_ROUTES[pe]) <= 1,
            f"one_mode_per_PE_{pe}_{m}"
        )

# ---- считаем среднюю калорийность режима для целевой категории ----
BEST_QUALITY = {}
for (pe, of, mode, tp, m) in QUALITY.keys():
    if TP_TO_CATEGORY[tp] == SUBJECT_QUALITY:
        BEST_QUALITY[(pe, of, mode, m)] = QUALITY[(pe, of, mode, tp, m)]

# ---- считаем выход по целевой категории ----
BEST_VOLUME = {}
for (pe, of, mode, tp, m) in YIELD.keys():
    if TP_TO_CATEGORY[tp] == SUBJECT_VOLUME:
        BEST_VOLUME[(pe, of, mode, m)] = YIELD[(pe, of, mode, tp, m)]

# ---- целевая функция ----

obj = 0

# 1) оптимизация качества
if OPTIMIZE_QUALITY == 1:
    model.sense = pulp.LpMaximize
    obj = pulp.lpSum(
        y[(pe, of, mode, m)] * BEST_QUALITY.get((pe, of, mode, m), 0)
        for pe in PEs
        for (of, mode) in PE_ROUTES[pe]
        for m in MONTHS
    )

# 2) оптимизация объёма
elif OPTIMIZE_VOLUME == 1:
    model.sense = pulp.LpMaximize
    obj = pulp.lpSum(
        y[(pe, of, mode, m)] * BEST_VOLUME.get((pe, of, mode, m), 0)
        for pe in PEs
        for (of, mode) in PE_ROUTES[pe]
        for m in MONTHS
    )

# 3) ничего не оптимизируем → просто feasible / минимизация себестоимости
else:
    model.sense = pulp.LpMinimize
    obj = pulp.lpSum(
        x_in[(pe, of, mode, m)] * (
            COST_PE_TO_OF.get((pe, of, m), 0) +
            VAR_ENRICH.get((of, m), 0)
        )
        for (pe, of, mode, m) in x_in.keys()
    )

# ---- tie-breaker по себестоимости ----
if SWITCHES.get("минимизация_себестоимости", 0) == 1 and (OPTIMIZE_QUALITY or OPTIMIZE_VOLUME):
    EPS = 1e-6
    cost_expr = pulp.lpSum(
        x_in[(pe, of, mode, m)] * (
            COST_PE_TO_OF.get((pe, of, m), 0) +
            VAR_ENRICH.get((of, m), 0)
        )
        for (pe, of, mode, m) in x_in.keys()
    )
    obj = obj - EPS * cost_expr

model.setObjective(obj)

print("✅ Новая objective установлена — режим выбирается по качеству/объёму правильно.")




✅ Новая objective установлена — режим выбирается по качеству/объёму правильно.


In [11]:
# === ЯЧЕЙКА 9. Решение задачи ===

solver = pulp.PULP_CBC_CMD(msg=1)  # msg=1 → выводит прогресс решения
result_status = model.solve(solver)

print("\n✅ Оптимизация завершена.")
print("Статус решения:", pulp.LpStatus[result_status])
print("Оптимальное значение objective:", pulp.value(model.objective))



✅ Оптимизация завершена.
Статус решения: Optimal
Оптимальное значение objective: 487661.2324344283


In [16]:
# === ЯЧЕЙКА 10. Формируем итоговый результат ===

rows = []

for pe in PEs:
    for (of, mode) in PE_ROUTES[pe]:
        for m in MONTHS:

            sent = x_in[(pe, of, mode, m)].value()
            if sent is None or sent <= 1e-9:
                continue  # ничего не отправили — пропускаем

            # по каждому ТП считаем выход и калорийность
            for tp in TPs:

                key = (pe, of, mode, tp, m)

                vol = YIELD.get(key, 0) * sent
                cal = QUALITY.get(key, 0)

                if vol <= 1e-9:
                    continue  # не производился данный ТП

                rows.append({
                    "месяц": m,
                    "ПЕ": pe,
                    "ОФ": of,
                    "отправлено, т": round(sent, 6),
                    "режим": mode,
                    "вид ТП": tp,
                    "объём ТП, т": round(vol, 6),
                    "калорийность ТП, ккал/кг": cal
                })

df_result = pd.DataFrame(rows).sort_values(
    ["месяц", "ПЕ", "ОФ", "режим", "вид ТП"]
)

print("✅ Итоговая таблица сформирована.")
df_result



KeyError: 'месяц'

In [ ]:
from pathlib import Path

output_path = Path.cwd() / "result_optimization.xlsx"
df_result.to_excel(output_path, index=False)

print("✅ Файл сохранён:", output_path)


In [17]:
print("Найдено строк результатов:", len(rows))

Найдено строк результатов: 0
